<a href="https://colab.research.google.com/github/ImHyunSeok/lim_in_aiffel5th/blob/main/%EB%85%B8%EB%93%9C%ED%95%99%EC%8A%B5%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B87/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%AA%A8%EB%8D%B8%ED%8A%9C%EB%8B%9D%EA%B3%BC%EB%B0%B0%ED%8F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


케라스튜너 설치

In [16]:
!mkdir /contet/drive/MyDrive/mlops_cifar10
!pip install keras-tuner

mkdir: cannot create directory ‘/contet/drive/MyDrive/mlops_cifar10’: No such file or directory


라이브러리 임포트, 데이터셋 로드

In [17]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [18]:
(x_train, y_train), (x_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [19]:
#정규화, 원핫 인코딩
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
y_train.shape

(50000, 10)

In [20]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size =0.2)
print(x_train.shape , x_val.shape)

(40000, 32, 32, 3) (10000, 32, 32, 3)


케라스 튜너

In [21]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, x, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(x, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        x_val, y_val = validation_data
        eval_scores = model.evaluate(x_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

모델 구성

In [28]:
def build_cifar10_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = x_train.shape[1:], name = 'inputs'))

    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
      model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_units_layer1_{i}', min_value=32, max_value=128, step=32),
                                      (3,3), activation='relu'))
      model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
      model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_units_layer2_{i}', min_value=32, max_value=128, step=32),
                                      (3,3), activation='relu'))
      model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
      model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_units_layer3_{i}', min_value=32, max_value=128, step=32),
                                      (3,3), activation='relu'))
      model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

하이어파라미터 찾기

In [29]:

class CustomTuner(kt.Tuner):

    def run_trial(self, trial, *args, **kwargs):
        # You can add additional customization here
        return super(CustomTuner, self).run_trial(trial, *args, **kwargs)

# Set up the custom tuner
my_keras_tuner = CustomTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('val_accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_cifar10_model,
    overwrite=True,
    project_name='cifar10_keras_tuner'
)

# Now, the search should work without errors
my_keras_tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=3)

Trial 10 Complete [00h 00m 00s]

Best val_accuracy So Far: 0.6589000225067139
Total elapsed time: 00h 24m 42s


In [30]:
#모델 확인

best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=1)[0]
model = build_cifar10_model(best_hps)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 15, 15, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 6, 6, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 96)          110688    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 2, 2, 96)         

In [32]:
model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val))
model.evaluate(x_test, y_test)

Epoch 1/20
1250/1250 [==============================] - 144s 114ms/step - loss: 1.5511 - accuracy: 0.4345 - val_loss: 1.3284 - val_accuracy: 0.5296
Epoch 2/20
1250/1250 [==============================] - 140s 112ms/step - loss: 1.1628 - accuracy: 0.5881 - val_loss: 1.0939 - val_accuracy: 0.6158
Epoch 3/20
1250/1250 [==============================] - 140s 112ms/step - loss: 0.9976 - accuracy: 0.6494 - val_loss: 1.0373 - val_accuracy: 0.6396
Epoch 4/20
1250/1250 [==============================] - 140s 112ms/step - loss: 0.8858 - accuracy: 0.6889 - val_loss: 0.9905 - val_accuracy: 0.6575
Epoch 5/20
1250/1250 [==============================] - 140s 112ms/step - loss: 0.8058 - accuracy: 0.7186 - val_loss: 1.0542 - val_accuracy: 0.6426
Epoch 6/20
1250/1250 [==============================] - 140s 112ms/step - loss: 0.7364 - accuracy: 0.7437 - val_loss: 0.8999 - val_accuracy: 0.6896
Epoch 7/20
1250/1250 [==============================] - 140s 112ms/step - loss: 0.6725 - accuracy: 0.7633 - val_

[1.4325824975967407, 0.6974999904632568]

In [33]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 12s 40ms/step - loss: 1.4326 - accuracy: 0.6975


[1.4325824975967407, 0.6974999904632568]

In [34]:
#최적모델저장
model.save('/content/drive/MyDrive/mlops_cifar10/best_model/1')

In [35]:
#최적모델불러오기

load_path = '/content/drive/MyDrive/mlops_cifar10/best_model/1'
best_model = tf.keras.models.load_model(load_path)

In [36]:
#최적 모델 요약
best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 15, 15, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 6, 6, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 96)          110688    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 2, 2, 96)         

In [37]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [38]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [39]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

{'serving_default': {'inputs': ['inputs'], 'outputs': ['dense_1']}}


In [40]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite